In [ ]:
# Basic Library
import random
from tqdm import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import torch

import numdifftools as nd
from scipy.optimize import minimize
import time


# normalize
def normalize(belief):   return belief/sum(belief)

# logistic
def sigmoid(x): return 1 / (1 + np.exp(-x))
def sigmoid_inv(x): return np.log(x) - np.log(1-x)

# log of Gaussian
def log_gaussian(x, mean, cov_inv):
    return -(1/2) * (x - mean).T @ (cov_inv) @ (x - mean)

# f
def LN(theta, x):
    return sigmoid(theta.T@x[:-1] + x[-1])

def MIRT(theta, x):
    return np.prod([sigmoid(theta[i]+x[i]) for i in range(len(x))])

class Record():
    def __init__(self):
        self.typs = ['MAP', 'nHess', 'f', 'true_f', 'time', 'R_L2']
        self.data = { typ:[] for typ in self.typs }

    def merge(self, subRecord):
        for typ in self.typs:
            self.data[typ].append(subRecord.data[typ])

# Algorithm
class GB():
    def __init__(self, model):
        self.f = staticmethod(model)
        
    def query(self, Belief, queries):
        MAP = Belief.MAP
        val, vec = np.linalg.eig(Belief.nHess)
        vec_min = vec[:,np.argmin(val)]
        delta = 1e-4
        idx = np.argmax(np.array([(self.f(MAP + delta*vec_min,queries[q_idx]) - self.f(MAP - delta*vec_min,queries[q_idx]))**2 for q_idx in range(len(queries))]))
        return idx

class EB() :
    def __init__(self, model):
        self.f = staticmethod(model)
        
    def query(self, Belief, queries):
        MAP = Belief.MAP
        min_ent = 1e9
        idx = 0
        for q_idx, x in enumerate(queries):
            f_map_x = self.f(MAP, x)
            det_1 = np.linalg.det(Belief.pseudo_bayesian_update(x, 1))
            det_0 = np.linalg.det(Belief.pseudo_bayesian_update(x, 0))
            ent = f_map_x * np.log(det_1) + (1 - f_map_x) * np.log(det_0)
            if min_ent > ent:
                min_ent = ent
                idx = q_idx
        return idx
            
class Belief():
    """
    Class Representation of Bayesian Belief.
    """
    def __init__(self, f, dim, MAP_0, nHess_0):
        self.dim = dim # dimension
        self.f = f # set correct rate function

        self.MAP_0 = MAP_0
        self.nHess_0 = nHess_0 # initial Hessian
        self.par_0 = {'MAP_0' : self.MAP_0, 'nHess_0' : self.nHess_0}
        
        self.LPrior = lambda theta : log_gaussian(theta, mean=self.MAP_0, cov_inv=self.nHess_0)
        self.initialize(self.MAP_0, self.nHess_0)

    def initialize(self, MAP_0, nHess_0): # clear history
        self.t, self.MAP, self.nHess = 0, MAP_0, nHess_0
        self.LL_t = [self.LPrior] # Log Likelihood at step t
 
        self.Record = Record()
        self.Record.data['MAP'].append(self.MAP)
        self.Record.data['nHess'].append(self.nHess)

    def NLPosterior(self, theta):
        return -sum(LL(theta) for LL in self.LL_t)

    def bayesian_update(self, x, y):
        LL = lambda theta : np.log( self.f(theta, x) if y==1 else 1-self.f(theta, x) ) # Log Likelihood
        self.LL_t.append(LL)

        func = self.NLPosterior
        gradient = nd.Gradient(func)
        hessian = nd.Hessian(func)

        # Laplace Approximation
        opt_response = minimize(func, self.MAP, jac=gradient, hess=hessian, method='trust-ncg')
        self.MAP = opt_response.x
        self.nHess = hessian(self.MAP)
        
        self.t += 1

        self.Record.data['MAP'].append(self.MAP)
        self.Record.data['nHess'].append(self.nHess)
        
    def pseudo_bayesian_update(self, x, y):
        LL = lambda theta : np.log( self.f(theta, x) if y==1 else 1-self.f(theta, x) ) # Log Likelihood
        func = self.NLPosterior
        gradient = nd.Gradient(func)
        hessian = nd.Hessian(func)
        opt_response = minimize(func, self.MAP, jac=gradient, hess=hessian, method='trust-ncg')
        nHess = hessian(opt_response.x)
        return nHess

class Experiment():
    def __init__(self, Algo, dim, f, num_t, MAP_0, nHess_0, R, P, Q, PQI):
        self.Algo = Algo
        self.dim = dim
        self.f = f
        self.num_t = num_t
        self.MAP_0 = MAP_0
        self.nHess_0 = nHess_0
        
        self.R = R
        self.P = P
        self.Q = Q
        self.PQI = PQI
    
    def do(self, populate_idx) :
        Data = []
        for i in tqdm(populate_idx) :
            
            self.Belief = Belief(self.f, self.dim, self.MAP_0, self.nHess_0)
            
            if self.num_t < len(self.PQI[i]) :
                self.true_theta = self.P[i]
                
                self.Belief.initialize(self.MAP_0, self.nHess_0)
                while self.Belief.t < self.num_t:
                    start_time = time.time()
                    print(self.Belief.t)
                    # Query suggestion by Algorithm
                    idx = self.Algo.query(self.Belief, self.Q[self.PQI[i]])
                    self.Belief.Record.data['true_f'].append(true_f := self.Belief.f(self.true_theta, self.Q[self.PQI[i][idx]]))
                    self.Belief.Record.data['f'].append(self.Belief.f(self.Belief.MAP, self.Q[self.PQI[i][idx]]))
                    
                    self.Belief.bayesian_update(self.Q[self.PQI[i][idx]], self.R[i][self.PQI[i][idx]])
                    self.PQI[i].pop(idx)
                    
                    end_time = time.time()
                    self.Belief.Record.data['time'].append(end_time - start_time)
            Data.append(self.Belief.Record)
        return Data
    
dim = 4
num_t = 50

models = [LN]

R = np.load("LN_dataset.npy")
# R = np.load("MIRT_Dataset.npy")

n_p, n_q = R.shape

obs = ~np.isnan(R)

possible_query_idx = []
for i in range(obs.shape[0]):
    true_indices = np.where(obs[i])
    possible_query_idx.append(true_indices[0])
    
PQIs = []

for i in range(len(possible_query_idx)) :
    PQIs.append(possible_query_idx[i].tolist())
PQI = PQIs

P = np.load("Theta_LN_4.npy")
Q = np.load("Query_LN_4.npy")
#P = np.load("Theta_MIRT_4.npy")
#Q = np.load("Query_MIRT_4.npy")

mean = np.mean(P,axis=0)
cov = np.cov(P.T)
MAP_0 = mean
n_Hess0 = np.linalg.inv(cov)


populate_indexes = np.load("LN_populate.npy")
populate_idx = populate_indexes[:50]

for model in models :
    #algorithm = EB(model)
    algorithm = GB(model)
    exp = Experiment(algorithm, dim, model, num_t,MAP_0, n_Hess0, R, P, Q, PQI)
    Data = exp.do(populate_idx)

Data = np.array(Data)
np.save("LN_" + str(dim) +"_GB.npy",Data)